## 星等转换为Flux单位

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.time import Time 
namelist = ['2003MA3','2010CO1','2010DG77','2010EX11','2010FC81','2010FH81','2010GS7','2010JN71','2010KX7','2010PW58','2010XP69','2019DD2']
for i in range(len(namelist)):
    name = namelist[i].lower()
    rawdata = pd.read_csv(f'{name}.csv')
    data = rawdata.loc[:,['mjd','w1mpro','w2mpro','w3mpro','w4mpro','ph_qual','sso_flg']]
    data.insert(loc=len(data.columns),column='date',value='no')
    #data['mjd'] = 2400000.5+data['mjd']
    for i in tqdm(range(len(data))):  
        t = Time(data['mjd'][i],format='mjd',scale='utc')
        t.format = 'iso'
        data['date'][i] = t.value
    #data.to_csv(f'{name}.txt',sep='\t',index=False)

  0%|          | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_181301/3363799465.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'][i] = t.value
100%|██████████| 23/23 [00:00<00:00, 2324.89it/s]


In [37]:
for i in range(len(namelist)):
    name = namelist[i].lower()
    rawdata = pd.read_csv(f'{name}.csv')
    data = rawdata.loc[:,['mjd','w1mpro','w2mpro','w3mpro','w4mpro','ph_qual','sso_flg']]
    for x in data.index:
        if data.loc[x, 'sso_flg'] ==0:
            data.drop(x, inplace = True)
    data.to_csv(f'New_{name}.txt',sep='\t',index=False)


In [106]:

df = pd.read_table('mpc_final_sort.txt')
df = df.loc[:229]
for i in range(len(df)):
    if df['PHA'][i]=='Potentially':
        df['PHA'][i] = 'Yes'
    else:
        df['PHA'][i] = 'No'
df = df.sort_values(by=['PHA','time','Diam'],ascending=False,ignore_index=True)
df.to_excel('PHA.xlsx')

/tmp/ipykernel_7561/1341410745.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PHA'][i] = 'No'
/tmp/ipykernel_7561/1341410745.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PHA'][i] = 'Yes'


## M2Flux


In [58]:
import numpy as np
import matplotlib.pyplot as plt
#-------initial para-------------------------
#Fv0 of W1 ,W2, W3, W4
Fv0 = [-1,306.682,170.663,29.045,8.284]
#Color corrections  Col[Bv=1:8=(100,141,200,283,400,566,800,1131),fc=1:4=(W1,W2,W3,W4)]
Fc0 = np.zeros((1,8))#W1,W2,W3,W4
Fc1 = np.array([-1,17.2062,3.9096,2.6588,1.0032]) #Bv=100
Fc2 = np.array([-1,4.0882,1.9739,1.4002,0.9852])  #Bv=141
Fc3 = np.array([-1,2.0577,1.3448,1.0006,0.9833])  #Bv=200
Fc4 = np.array([-1,1.3917,1.1124,0.8791,0.9865])  #Bv=283
Fc5 = np.array([-1,1.1316,1.0229,0.8622,0.9903])  #Bv=400
Fc6 = np.array([-1,1.0263,0.9919,0.8833,0.9935])  #Bv=566
Fc7 = np.array([-1,0.9884,0.9853,0.9125,0.9958])  #Bv=800
Fc8 = np.array([-1,0.9801,0.9877,0.9386,0.9975])  #Bv=1131
Fc = [Fc0,Fc1,Fc2,Fc3,Fc4,Fc5,Fc6,Fc7,Fc8]
Fc = np.array(Fc)
Bv = 4
Const = np.array([-1,Fv0[1]/Fc[Bv][1],Fv0[2]/Fc[Bv][2],Fv0[3]/Fc[Bv][3],Fv0[4]/Fc[Bv][4]])
#-------read file----------------------------
for i in range(len(namelist)):
    name = namelist[i].lower()
    #df = pd.read_table(f'{name}.txt')
    df = pd.read_table(f'New_{name}.txt')
    #data of W1,W2,W3,W4
    dat_w1 = np.array(df['w1mpro'])
    #dat_w1= np.hstack([dat_w1,dat_w1*0.1])
    dat_w2 = np.array(df['w2mpro'])
    #dat_w2= np.hstack([dat_w2,dat_w2*0.1])
    dat_w3 = np.array(df['w3mpro'])
    #dat_w3= np.hstack([dat_w3,dat_w3*0.1])
    dat_w4 = np.array(df['w4mpro'])
    #dat_w4= np.hstack([dat_w4,dat_w4*0.1])
    #time in MJD
    mjd =  np.array(df['mjd'])
    #dat w1
    dat_fluxw1 = dat_w1
    dat_fluxw1 = Const[1]*np.power(10,-dat_w1/2.5)*1000 
    #dat w2
    dat_fluxw2 = dat_w2
    dat_fluxw2 = Const[1]*np.power(10,-dat_w2/2.5)*1000 
    #dat w3
    dat_fluxw3 = dat_w3
    dat_fluxw3 = Const[1]*np.power(10,-dat_w3/2.5)*1000 
    print(f'{Const[1]}*{np.power(10,-dat_w3[0]/2.5)}*1000={dat_fluxw3[0]}')
    #dat w4
    dat_fluxw4 = dat_w4
    dat_fluxw4 = Const[1]*np.power(10,-dat_w4/2.5)*1000 
    dat_w3w4 = np.concatenate([dat_fluxw3,dat_fluxw4],axis=0)
    np.savetxt('w34_'+name + '.txt',dat_w3w4)

x = range(len(dat_w3w4))
y = dat_w3w4
plt.plot(x,y,'r.',linewidth=1.2)
#plt.errorbar(x,y,fmt="bo:",yerr=y*0.1,linewidth=0.6)
plt.show()

220.3650211970971*4.5331490614641154e-05*1000=9.989474890191406
220.3650211970971*0.00019534396563982562*1000=43.04697712894518


/tmp/ipykernel_181301/1874399844.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Fc = np.array(Fc)


IndexError: index 0 is out of bounds for axis 0 with size 0